# TP4, INF8225 2025, Projet


## Imports

In [1]:
from IPython.display import clear_output

%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121
%pip install numpy
%pip install swig
%pip install box2d
%pip install pygame
%pip install gymnasium
%pip install "gymnasium[box2d]"
%pip install matplotlib
%pip install wandb

clear_output()

In [ ]:
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt
from IPython.display import HTML
import matplotlib.animation as animation
import torch
import torch.nn as nn
from torch.distributions import Normal, MultivariateNormal, Categorical
import torchvision.transforms as T
import pygame
import time
import os
import wandb
from IPython.display import clear_output
import math
from collections import namedtuple, deque

### Initialisation

In [65]:
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(torch.version.cuda)

if torch.cuda.is_available():
	print(torch.cuda.get_device_name(0))

True
12.1
NVIDIA GeForce GTX 1050


## Data Declaration

In [66]:
# Inspired by : https://github.com/pangyyen/carRacing-DeepRL/blob/main/ppo/ppo.ipynb

env = gym.make("CarRacing-v3", render_mode="rgb_array", domain_randomize=False)
print("Observation space: ", env.observation_space) # (low, high, shape, dtype)
print("Action space: ", env.action_space)

SEED = 42

observation, info = env.reset(seed=SEED)

def show_animation():
	show_animation_frames(env.render())

def show_animation_frames(frames):
	fig = plt.figure(figsize=(7, 5))
	plt.axis('off')
	im = plt.imshow(frames[0])

	def animate(i):
		im.set_data(frames[i])
		return im,

	anim = animation.FuncAnimation(fig, animate, frames=len(frames), repeat=False)
	plt.close(fig)
	display(HTML(anim.to_jshtml()))

def show_current_frame(env, data):
	frame = env.render()
	fig, _ = plt.subplots()
	r = fig.canvas.get_renderer()
	plt.imshow(frame)
	plt.axis('off')
	texts = []
	size_used = 0
	for i, key in enumerate(data):
		text = plt.text(0, 0, f'{key}: {data[key]}', fontsize=12, color='black', backgroundcolor='white', ha="center")
		size_used += text.get_window_extent(renderer=r).width
		texts.append(text)
	split = (700 - size_used) / (len(data) + 1)
	next_position = split
	for t in texts:
		t.set_position((next_position, 0))
		next_position = next_position + t.get_window_extent(renderer=r).width + split
	clear_output(wait=True)
	plt.show()

def skip_zooming(env):
	no_action = 0
	if env.action_space.shape == (3,):
		no_action = np.array([0.0, 0.0, 0.0])

	for i in range(50):
		observation, _, terminated, truncated, info = env.step(no_action)

		if terminated or truncated:
			observation, info = env.reset()
			break
	return observation, info

Observation space:  Box(0, 255, (96, 96, 3), uint8)
Action space:  Box([-1.  0.  0.], 1.0, (3,), float32)


## Implementation

### Human Testing

In [ ]:
env = gym.make("CarRacing-v3", render_mode="human")
obs = env.reset()[0]

pygame.init()
win = pygame.display.set_mode((1000, 1000))
pygame.display.set_caption("Keyboard Controller")

clock = pygame.time.Clock()

running = False # TODO Set to TRUE if you want to play manually
action = np.array([0.0, 0.0, 0.0])  # [steer, gas, brake]

def process_keys():
    keys = pygame.key.get_pressed()
    steer = 0.0
    gas = 0.0
    brake = 0.0

    if keys[pygame.K_LEFT]:
        steer = -1.0
    elif keys[pygame.K_RIGHT]:
        steer = 1.0

    if keys[pygame.K_UP]:
        gas = 1.0
    if keys[pygame.K_DOWN]:
        brake = 1.0

    return np.array([steer, gas, brake])

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    action = process_keys()
    obs, reward, terminated, truncated, _ = env.step(action)
    done = terminated or truncated

    if done:
        obs = env.reset()[0]

    clock.tick(60)  # Limit to 60 FPS

env.close()
pygame.quit()